In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
# data = pd.read_csv('../input/spam.csv',encoding='latin-1')
data = pd.read_csv('./spam.csv',encoding='latin-1')

data = data.drop(["Unnamed: 2", "Unnamed: 3", "Unnamed: 4"], axis=1)
data = data.rename(columns={"v1":'label', "v2":'text'})
print(data.head())
tags = data["label"]
texts = data["text"]

  label                                               text
0   ham  Go until jurong point, crazy.. Available only ...
1   ham                      Ok lar... Joking wif u oni...
2  spam  Free entry in 2 a wkly comp to win FA Cup fina...
3   ham  U dun say so early hor... U c already then say...
4   ham  Nah I don't think he goes to usf, he lives aro...


In [2]:
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation
from keras.layers import Embedding
from keras.layers import Conv1D, GlobalMaxPooling1D
from keras.preprocessing.text import Tokenizer
from keras.preprocessing import sequence
from sklearn.preprocessing import LabelEncoder
from keras import metrics

num_max = 1000
le = LabelEncoder()
tags = le.fit_transform(tags)
tok = Tokenizer(num_words=num_max)
tok.fit_on_texts(texts)
mat_texts = tok.texts_to_matrix(texts,mode='count')

from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test = train_test_split(texts,tags, test_size = 0.3)
mat_texts_tr = tok.texts_to_matrix(x_train,mode='count')
mat_texts_tst = tok.texts_to_matrix(x_test,mode='count')

max_len = 100
x_train = tok.texts_to_sequences(x_train)
x_test = tok.texts_to_sequences(x_test)
cnn_texts_mat = sequence.pad_sequences(x_train,maxlen=max_len)
max_len = 100
cnn_texts_mat_tst = sequence.pad_sequences(x_test,maxlen=max_len)


Using TensorFlow backend.


In [3]:
from keras.layers import LSTM

max_features = 5000
maxlen = 400
batch_size = 32
embedding_dims = 50
filters = 250
kernel_size = 3
hidden_dims = 250
epochs = 2

In [4]:
def check_model(model,xtr,ytr,xts,yts):
    model.fit(xtr,ytr,batch_size=32,epochs=10,verbose=1,validation_split=0.3)
    print(' ')
    model.evaluate(xts,yts)

In [5]:
def get_LSTM_v4():    
    model = Sequential()
    model.add(Embedding(max_features, 128))
    model.add(LSTM(128, dropout=0.2, recurrent_dropout=0.2,activation='relu'))
    model.add(Dense(1, activation='sigmoid'))
    model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['acc',metrics.binary_accuracy])

    print('Train...')
    return model

In [6]:
m = get_LSTM_v4()
check_model(m,cnn_texts_mat,y_train,cnn_texts_mat_tst ,y_test)

Train...


/Users/drew/anaconda3/lib/python3.7/site-packages/tensorflow_core/python/framework/indexed_slices.py:424: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


Train on 2730 samples, validate on 1170 samples
Epoch 1/10
2730/2730 [==============================] - 29s 11ms/step - loss: 3038.4771 - acc: 0.8516 - binary_accuracy: 0.8516 - val_loss: 0.5300 - val_acc: 0.8769 - val_binary_accuracy: 0.8769
Epoch 2/10
2730/2730 [==============================] - 29s 11ms/step - loss: 91444371985.5141 - acc: 0.9374 - binary_accuracy: 0.9374 - val_loss: 0.3627 - val_acc: 0.9692 - val_binary_accuracy: 0.9692
Epoch 3/10
2730/2730 [==============================] - 29s 11ms/step - loss: 712038.0842 - acc: 0.9733 - binary_accuracy: 0.9733 - val_loss: 0.2463 - val_acc: 0.9778 - val_binary_accuracy: 0.9778
Epoch 4/10
2730/2730 [==============================] - 30s 11ms/step - loss: 325.5552 - acc: 0.9810 - binary_accuracy: 0.9810 - val_loss: 1.5619 - val_acc: 0.9786 - val_binary_accuracy: 0.9786
Epoch 5/10
2730/2730 [==============================] - 29s 11ms/step - loss: 173950292973.6938 - acc: 0.9795 - binary_accuracy: 0.9795 - val_loss: 2.5424 - val_acc